In [ ]:
# PowerShell MCP Server Usage Guide

This notebook demonstrates how to use the MCP PowerShell Exec Server for various PowerShell tasks.

## First, let's import the required libraries
import requests
import json

## Define helper functions to interact with the MCP server

def run_powershell(code, server_url="http://localhost:8000"):
    """Run a PowerShell command and get the result"""
    url = f"{server_url}/mcp/tools/run_powershell"
    payload = {"code": code}
    response = requests.post(url, json=payload)
    return response.text

def run_powershell_formatted(code, output_format="json", server_url="http://localhost:8000"):
    """Run a PowerShell command with formatted output"""
    url = f"{server_url}/mcp/tools/run_powershell_formatted"
    payload = {"code": code, "format": output_format}
    response = requests.post(url, json=payload)
    if output_format == "json":
        try:
            return json.loads(response.text)
        except json.JSONDecodeError:
            return response.text
    return response.text

## Example 1: Basic PowerShell Command
# Let's start with a simple PowerShell command to get the current date

result = run_powershell("Get-Date")
print(result)

## Example 2: Formatted Output
# Get a list of running processes and format as JSON

processes = run_powershell_formatted("Get-Process | Select-Object -First 5 Id, ProcessName, CPU", "json")
print(json.dumps(processes, indent=2))

## Example 3: Working with the File System
# List files in the current directory

files = run_powershell_formatted("Get-ChildItem -Path . | Select-Object Name, Length, LastWriteTime", "json")
print(json.dumps(files, indent=2))

## Example 4: System Information
# Get basic system information

system_info = run_powershell("""
$computerSystem = Get-CimInstance -Class Win32_ComputerSystem
$computerBIOS = Get-CimInstance -Class Win32_BIOS
$computerOS = Get-CimInstance -Class Win32_OperatingSystem

Write-Output "Computer Name: $($computerSystem.Name)"
Write-Output "Manufacturer: $($computerSystem.Manufacturer)"
Write-Output "Model: $($computerSystem.Model)"
Write-Output "OS: $($computerOS.Caption) $($computerOS.Version)"
""")
print(system_info)

## Example 5: Data Processing
# Process CSV data with PowerShell

csv_processing = run_powershell_formatted("""
$data = ConvertFrom-Csv @"
Date,Product,Category,Amount
2025-04-01,Laptop,Electronics,1299.99
2025-04-02,Monitor,Electronics,349.99
2025-04-03,Desk Chair,Furniture,189.50
2025-04-05,Coffee Maker,Appliances,79.99
"@

$data | Group-Object Category |
    Select-Object @{Name='Category';Expression={$_.Name}},
                  @{Name='TotalSales';Expression={($_.Group | Measure-Object -Property Amount -Sum).Sum}},
                  @{Name='Count';Expression={$_.Count}}
""", "json")
print(json.dumps(csv_processing, indent=2))

## Example 6: Error Handling
# Demonstrate how the server handles errors

try:
    error_result = run_powershell("Get-NonExistentCommand")
    print(error_result)
except Exception as e:
    print(f"Error: {e}")